<a href="https://colab.research.google.com/github/DeepaManjunath/Assignment1/blob/master/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q '/content/gdrive/My Drive/EIP4/hvc_data.zip'
# look for `hvc_annotations.csv` file and `resized` dir
%ls '/content/gdrive/My Drive/EIP4/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
hvc_data/  hvc_data.zip


In [3]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation , Dropout
from keras.layers import AveragePooling2D, Input, Flatten ,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model


Using TensorFlow backend.


In [4]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [5]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,imagegen=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.batch_size = batch_size
        #self.labels = labels
        #self.list_IDs = list_IDs
        #self.n_channels = n_channels
        #self.n_classes = n_classes
        #self.shuffle = shuffle
        
        if imagegen==True:
        
          featurewise_center=False,
          samplewise_center=False,
          featurewise_std_normalization=False,
          samplewise_std_normalization=False,
          zca_whitening=False,
          zca_epsilon=1e-06,
          rotation_range=0.0,
          width_shift_range=0.1,
          height_shift_range=0.1,
          brightness_range=None,
          shear_range=0.0,
          zoom_range=0.0,
          channel_shift_range=0.0,
          fill_mode='nearest',
          cval=0.0,
          horizontal_flip=True,
          vertical_flip=False,
          rescale=None,
          preprocessing_function=get_random_eraser(v_l=0, v_h=1),
          data_format=None,
          validation_split=0.0

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))


        
    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
   

In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [9]:
len(train_df)

11537

In [10]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
4198,resized/4199.jpg,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0
6374,resized/6375.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
1307,resized/1308.jpg,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
11111,resized/11113.jpg,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0
4429,resized/4430.jpg,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,imagegen=True,shuffle=True)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=True,imagegen=False)

In [12]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 24
num_classes = 8

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 9

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, num_classes=8):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    outputs = (x)
   # outputs = Dense(num_classes,
                    #activation='softmax',
                    #kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



In [14]:
#from classification_models.resnet import ResNet18, preprocess_input
import keras.applications.resnet 
from keras.applications.inception_v3 import InceptionV3

from keras.applications.resnet import ResNet50

backbone = InceptionV3(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)
#backbone = resnet_v1((224,224,3),8)
  


neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = BatchNormalization(momentum=0.355)(in_layer)
    neck = Dropout(0.0712)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = BatchNormalization(momentum=0.355)(in_layer)
    neck = Dropout(0.0712)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = BatchNormalization(momentum=0.355)(in_layer)
    neck = Dropout(0.0712)(in_layer)
    neck = Dense(28, activation="relu")(neck)
    
    
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="sigmoid", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
import numpy as np
import tensorflow as tf
import keras


In [0]:
EPOCHS=50
LEARNING_RATE=0.01
BATCH_SIZE1=32
BATCH_SIZE2=64
MOMENTUM=0.9

In [0]:
len_train=len(train_df)
batches_per_epoch = len_train//BATCH_SIZE1 


In [18]:
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS)], [0.001428, LEARNING_RATE, 0.001428] )[0]
print(lr_schedule)

#lr_schedule = lambda t: np.interp([t],[0,(EPOCHS+1)//5, (EPOCHS-6)],[0.4, LEARNING_RATE, 0.4])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE1
#opt.apply_gradients( global_step=global_step)
#opt = tf.train.MomentumOptimizer(lr_func,momentum=MOMENTUM,use_nesterov=False)

<function <lambda> at 0x7fd69b2d8730>


In [0]:
#t = time.time()
#test_set = tf.data.Dataset.from_tensor_slices((val_df)).batch(BATCH_SIZE)

def lr_schedule1(epoch):
    for epoch1 in range((EPOCHS)):
    #if epoch<=24:
      
         if epoch==epoch1:
          break
     
   
   # tf.keras.backend.set_learning_phase(1)
         opt = tf.train.MomentumOptimizer(lr_func,momentum=MOMENTUM,use_nesterov=False)
         #opt1 = SGD(lr=lr_schedule(epoch+1),momentum=0.9)
        #if epoch==epoch1:
         # break
          
    print('epoch:', epoch, 'lr:', lr_schedule(epoch))
    return(lr_schedule(epoch))

      
     
    #tf.keras.backend.set_learning_phase(0)
    
      #  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1))
  #return (lr_schedule(epoch+1))
    #else:
   
    #tf.keras.backend.set_learning_phase(1)
     #   opt = tf.train.MomentumOptimizer(lr_func1,momentum=MOMENTUM,use_nesterov=False)
      #  opt = SGD(lr=lr_schedule(epoch+1),momentum=0.9)
        #model.compile(
         #           optimizer=opt,
          #          loss="categorical_crossentropy", 
           #         loss_weights=loss_weights, 
            #        metrics=["accuracy"]
             #       )
        #model.fit_generator(
         #            generator=train_gen,
          #           validation_data=valid_gen,
           #          use_multiprocessing=True,
            #         workers=6, 
             #        epochs=10,
              #       verbose=1,
      
               #      callbacks=callbacks
                #         )
        #LRFind.plot_loss()

       
    #tf.keras.backend.set_learning_phase(0)
       # print('epoch:', epoch+1, 'lr:', lr_schedule1(epoch+1))

In [27]:
 losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "binary_crossentropy",
 	"age_output": "binary_crossentropy",
 	"weight_output": "binary_crossentropy",
  "bag_output": "binary_crossentropy",
  "pose_output": "binary_crossentropy",
  "footwear_output": "binary_crossentropy",   
  "emotion_output": "binary_crossentropy",
       

 }
loss_weights = {"gender_output": 0.2, "image_quality_output": 0.3, "age_output": 0.5, "weight_output": 0.4,"bag_output":0.3,"pose_output":0.3,"footwear_output":0.3,"emotion_output":0.4}
model.summary()

opt = SGD(lr=lr_schedule1(0),momentum=0.75)
#opt = tf.train.MomentumOptimizer(lr=lr_schedule(epoch+1),momentum=MOMENTUM,use_nesterov=False)
#opt.( global_step=global_step)
model.compile(
    optimizer=opt,
    loss="binary_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:
import matplotlib.pyplot as plt
import keras.backend as K
from keras.callbacks import Callback


class LRFinder(Callback):
    
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=5.625, epochs=24):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        plt.show()
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.show()

In [0]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
#for i in range(EPOCHS):
#lr_scheduler = LearningRateScheduler(lr_schedule1)

In [0]:
#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS-6)], [0.4, LEARNING_RATE, 0.4])[0]
#global_step = tf.train.get_or_create_global_step()
#opt.apply_gradients(zip(grads, var), global_step=global_step

In [0]:
#from keras.callbacks import ModelCheckpoint,LearningRateScheduler

LRFind=LRFinder(min_lr=1e-5, max_lr=10, steps_per_epoch=5.625, epochs=50)
#lr_scheduler = LearningRateScheduler(lr_schedule)
#LRFind.on_tain_begin()
#for i in range(EPOCHS):
#callbacks = [LRFind]

In [28]:
#for i in range(EPOCHS):
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    #callbacks=callbacks
    )
#LRFind.plot_loss()


Epoch 1/50
360/360 [==============================] - 111s 308ms/step - loss: 1.3020 - gender_output_loss: 0.5529 - image_quality_output_loss: 0.5940 - age_output_loss: 0.4753 - weight_output_loss: 0.4260 - bag_output_loss: 0.5267 - footwear_output_loss: 0.4940 - pose_output_loss: 0.4803 - emotion_output_loss: 0.3872 - gender_output_acc: 0.7159 - image_quality_output_acc: 0.6975 - age_output_acc: 0.7815 - weight_output_acc: 0.8175 - bag_output_acc: 0.7317 - footwear_output_acc: 0.7538 - pose_output_acc: 0.7720 - emotion_output_acc: 0.8540 - val_loss: 1.3018 - val_gender_output_loss: 0.5814 - val_image_quality_output_loss: 0.5859 - val_age_output_loss: 0.4539 - val_weight_output_loss: 0.4220 - val_bag_output_loss: 0.5212 - val_footwear_output_loss: 0.5224 - val_pose_output_loss: 0.4969 - val_emotion_output_loss: 0.3796 - val_gender_output_acc: 0.6845 - val_image_quality_output_acc: 0.6828 - val_age_output_acc: 0.7922 - val_weight_output_acc: 0.8135 - val_bag_output_acc: 0.7270 - val_foo

In [0]:
model